In [250]:
import pickle

with open("Data\\feature_embeddings_50.bin", "rb") as f:
    embeddings = pickle.load(f)
with open("Data\\genre_probabilities_vector.bin", "rb") as f:
    genre_probabilities_vector = pickle.load(f)

In [ ]:
embeddings

In [56]:
with open("Data\\top_plots.bin", "rb") as f:
    top250_plots = pickle.load(f)

In [57]:
movie = top250_plots["The Truman Show"]

In [58]:
import spacy 

nlp = spacy.load("en_core_web_sm")

In [251]:
doc = nlp(movie)

open_word_classes = ["VERB", "NOUN", "ADJ"]

keywords_present = set()

for token in doc:
    if token.pos_ not in open_word_classes:
        continue
    
    keyword = token.lemma_
    if keyword in embeddings:
        keywords_present.add(keyword)

In [90]:
len(keywords_present)

233

In [92]:
import numpy as np

n = embeddings['crime'].shape[0]
#m = len(keywords_present)
movie_vec = np.ones(n)

for keyword in keywords_present:
    vec = embeddings[keyword]
    movie_vec *= vec

movie_vec *= genre_probabilities_vector

movie_vec
#movie_vec /= m

array([2.21436376e-018, 3.39006826e+006, 3.08860921e-131, 1.36218471e-034,
       1.93562266e-144, 3.90427106e-053, 5.38017200e-011, 5.36648167e-010,
       1.04939200e+010, 7.12594748e-007, 5.53214754e-109, 1.29382613e-129,
       2.66876626e-117, 1.83234483e+012, 4.33366895e+002, 2.16994033e-031,
       1.66442162e-013, 7.36895845e-082, 1.09325839e+001, 5.26937880e-033,
       9.58720960e-045])

In [93]:
with open("Data\\genre_master_list.bin", "rb") as f:
    genre_master_list = pickle.load(f)

ignore_genres = ["Musical", "Animation"]

for genre in ignore_genres:
    idx = genre_master_list.index(genre)
    movie_vec[idx] = 0

movie_vec

array([2.21436376e-018, 3.39006826e+006, 3.08860921e-131, 1.36218471e-034,
       1.93562266e-144, 3.90427106e-053, 5.38017200e-011, 5.36648167e-010,
       0.00000000e+000, 7.12594748e-007, 5.53214754e-109, 0.00000000e+000,
       2.66876626e-117, 1.83234483e+012, 4.33366895e+002, 2.16994033e-031,
       1.66442162e-013, 7.36895845e-082, 1.09325839e+001, 5.26937880e-033,
       9.58720960e-045])

In [51]:
import math

def hardmax(vector):
    sum_total = 0
    new = vector.copy()
    n = vector.shape[0]

    for i in range(n):
        sum_total += vector[i]
    
    new /= sum_total

    return new

def softmax(vector):
    exp_total = 0
    new = vector.copy()
    n = vector.shape[0]

    for i in range(n):
        exp = math.exp(vector[i])
        exp_total += exp
        new[i] = exp
    
    new /= exp_total

    return new

In [94]:
movie_vec = hardmax(movie_vec)
movie_vec

array([1.20848414e-030, 1.85012228e-006, 1.68560167e-143, 7.43409306e-047,
       1.05636180e-156, 2.13074733e-065, 2.93621701e-023, 2.92874554e-022,
       0.00000000e+000, 3.88897013e-019, 3.01915733e-121, 0.00000000e+000,
       1.45647330e-129, 9.99998150e-001, 2.36509028e-010, 1.18424015e-043,
       9.08354428e-026, 4.02159283e-094, 5.96643355e-012, 2.87575187e-045,
       5.23219851e-057])

In [96]:
idx_of_max = 0

for idx, val in enumerate(movie_vec):
    if val > movie_vec[idx_of_max]:
        idx_of_max = idx
print(idx_of_max)
print(genre_master_list[idx_of_max])

13
Adventure


In [109]:
def probable_genre(movie_vec, genre_master_list):
    idx_of_max = 0
    for idx, val in enumerate(movie_vec):
        if val > movie_vec[idx_of_max]:
            idx_of_max = idx
    return genre_master_list[idx_of_max]

In [158]:
def get_movie_vector(movie_plot, embeddings, nlp, genre_master_list, mult = True, hard = True):

    with open("Data\\genre_probabilities_vector.bin", "rb") as f:
        genre_probabilities_vector = pickle.load(f)
    
    doc = nlp(movie_plot)

    word_classes = ["VERB", "NOUN", "ADJ"]

    keywords_present = set()

    for token in doc:
        if token.pos_ not in word_classes:
            continue
        
        keyword = token.lemma_
        if keyword in embeddings:
            keywords_present.add(keyword)

    n = embeddings['crime'].shape[0]
    movie_vec = np.ones(n) if mult else np.zeros(n)

    for keyword in keywords_present:
        vec = embeddings[keyword]
        if mult:
            movie_vec *= vec
        else:
            movie_vec += vec
            

    movie_vec *= genre_probabilities_vector

    ignore_genres = ["Musical", "Animation"]

    for genre in ignore_genres:
        idx = genre_master_list.index(genre)
        movie_vec[idx] = 0

    final = hardmax(movie_vec) if hard else (movie_vec / len(keywords_present))

    return final


In [142]:
def get_movie_plot(ia, title, release_year):
    search_results = ia.search_movie(title)
    
    movieID = None
    # get first result
    if release_year == -1:
        movieID = search_results[0].movieID
    # get first result that matches year
    else:
        for result in search_results:
            if result['year'] == release_year:
                movieID = result.movieID
                break
    # get closest match
    if not movieID:
        closest = sorted(search_results, key=lambda x: abs(x['year'] - release_year))
        movieID = closest[0].movieID
    
    movie = ia.get_movie(movieID, info=['synopsis'])

    key = movie.infoset2keys['synopsis'][0]

    plots = movie[key]
    plot = ""

    for p in plots:
        plot += f"{p} "

    return plot

In [160]:
def mv(movie, year, mult=True, hard=True):
    import spacy
    import numpy as np
    from imdb import Cinemagoer

    nlp = spacy.load("en_core_web_sm")
    ia = Cinemagoer()

    with open("Data\\feature_embeddings.bin", "rb") as f:
        embeddings = pickle.load(f)
    with open("Data\\top_plots.bin", "rb") as f:
        top250_plots = pickle.load(f)
    with open("Data\\genre_master_list.bin", "rb") as f:
        genre_master_list = pickle.load(f)

    #movie = 'Trainspotting'
    for i in range(10):
        try:
            plot = get_movie_plot(ia, movie, year)
        except:
            continue
        break
    else:
        print(f"Could not get plot for {movie}")
        return

    
    v = get_movie_vector(plot, embeddings, nlp, genre_master_list, mult=mult, hard=hard)

    return v
    #pgen = probable_genre(v, genre_master_list)

    #print(f"{movie} is a {pgen} movie")

    #v

In [118]:
def get_user_liked_and_disliked_movies(user):
    pfile = f"Data\\{user}.txt" 

    all_movies = {}
    liked_movies = {}
    disliked_movies = {}

    with open(pfile) as p:
        lines = p.readlines()
        for line in lines:
            parts = line.strip().split(",")
            liked = parts[0][0]
            title = parts[0][1:]
            if len(parts) > 1:
                year = int(parts[1])
            else:
                year = -1
            if liked == "+":
                liked_movies[title] = year
            else:
                disliked_movies[title] = year
            all_movies[title] = year

    return all_movies, liked_movies, disliked_movies

In [119]:
all, liked, disliked = get_user_liked_and_disliked_movies("Trevor")

In [252]:
vectors = {}
L = len(genre_master_list)
liked_vector = np.zeros(L)
total_vector = np.zeros(L)

for movie, year in all.items():
    vec = mv(movie, year, mult=False, hard=False)
    if vec is None:
        continue
    vectors[movie] = vec
    total_vector += vec
    if movie in liked:
        liked_vector += vec

preferences = np.zeros(L)

for i in range(L):
    if total_vector[i] != 0:
        preferences[i] = liked_vector[i] / total_vector[i]

Could not get plot for Beavis and Butthead Do America


In [253]:
vectors

{'Inception': array([0.23151975, 0.26369735, 0.01567247, 0.09480617, 0.02145674,
        0.04457934, 0.64730786, 0.07006352, 0.        , 0.20655925,
        0.01569391, 0.        , 0.01438605, 0.24713666, 0.11273667,
        0.02853947, 0.11260639, 0.04055699, 0.14850717, 0.10812334,
        0.05716884]),
 'Gladiator': array([0.2250385 , 0.2752397 , 0.0288224 , 0.08778669, 0.01696574,
        0.04321343, 0.67351773, 0.0695331 , 0.        , 0.19632959,
        0.01479287, 0.        , 0.01285059, 0.27770521, 0.0762054 ,
        0.03225633, 0.09200071, 0.05847356, 0.14087776, 0.10628697,
        0.07336635]),
 'Interstellar': array([0.17004402, 0.29680461, 0.01687268, 0.07605833, 0.0186842 ,
        0.04601301, 0.58833839, 0.06884328, 0.        , 0.17998564,
        0.01174582, 0.        , 0.01455265, 0.31333388, 0.18093458,
        0.04147772, 0.12503651, 0.05116591, 0.15150319, 0.10027687,
        0.05392883]),
 'Shutter Island': array([0.23844791, 0.25231958, 0.01857272, 0.09518404, 0.

In [254]:
training_X = []
training_Y = []

for movie, vector in vectors.items():
    training_X.append(vector)
    if movie in liked:
        training_Y.append(1)
    else:
        training_Y.append(0)

In [255]:
from sklearn import tree

clsfr = tree.DecisionTreeClassifier()

clsfr.fit(training_X[2:-2], training_Y[2:-2])

DecisionTreeClassifier()

In [256]:
for movie, vector in vectors.items():
    prediction = clsfr.predict(vector.reshape(1, -1))
    print(prediction)

[1]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[1]


In [259]:
test = mv("Grand budapest hotel", -1)

clsfr.predict(test.reshape(1, -1))

array([1])

In [147]:
suite = mv("Grand budapest hotel", -1)

#probability = max(suite * preferences)

In [148]:
probability

np.float64(0.4316201415463337)

In [151]:
def will_i_like(movie, year):
    mvec = mv(movie, year)
    if mvec is None:
        print("Error")
        return -1
    prob = max(mvec * preferences)
    if prob >= 0.5:
        print("I think you will like this movie!")
        return 1
    else:
        print("I think you will not like this movie...")
        return 0

In [157]:
will_i_like("Interstellar", -1)

I think you will like this movie!


1